# <Pseudo_LiDAR> 
https://github.com/mileyan/Pseudo_Lidar

notebook maker : lee jiho (jiho264@naver.com)

PSMNet 내 구형 torchvision 이슈로, PSMNet은 상위 버전을 가져와서, 코드를 일부 수정해야함.
https://github.com/JiaRenChang/PSMNet

정리
1. PSMNet 알고리즘으로 depth map 만듦 (.png >>> .npy)
2. depth map을 point cloud로 만듦 (.npy >>> .bin)
3. frustum pointnet 알고리즘으로 데이터 전처리 실행. 원본 .npg + pseudo lidar의 output인 .bin 이용. (.png + .bin >>> .pickle)
4. frustum의 viewer code 들로 객체 감지와, dataset 전체의 객체 감지 통계 확인가능.

# 1. Overview
We provide the guidance and codes to train stereo depth estimator and 3D object detector using the KITTI object detection benchmark. We also provide our pre-trained models.

# 2. Usage

## 2.1 Dependencies
- Python 3.5+
- numpy, scikit-learn, scipy
- KITTI 3D object detection dataset
- CUDA, Torch

In [ ]:
'''
1. cuda 11.3.1 install
    wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
    sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
    wget https://developer.download.nvidia.com/compute/cuda/11.3.1/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.1-465.19.01-1_amd64.deb
    sudo dpkg -i cuda-repo-ubuntu2004-11-3-local_11.3.1-465.19.01-1_amd64.deb
    sudo apt-key add /var/cuda-repo-ubuntu2004-11-3-local/7fa2af80.pub
    sudo apt-get update
    sudo apt-get -y install cuda
2. maybe have dpkg error (1) 
    cd /var/lib/dpkg
    sudo rm -r info
    sudo mkdir info
3. retry install CUDA
    sudo apt-get -y install cuda
4. .bach edit. 
    sudo gedit /etc/bash.bashrc    
        >>> add to last line
        export PATH=$PATH:/usr/local/cuda-11.3/bin
        export CUDADIR=/usr/local/cuda-11.3
        export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-11.3/lib64
    source /etc/bash.bashrc
5. check my driver and remove (if display resolution 640*480)
    dpkg -l | grep -i nvidia
    >>> ex) nvidia-driver-465
    sudo apt remove nvidia-driver-465
6. check and install 
    ubuntu-drivers devices
    sudo apt install nvidia-driver-515
    sudo reboot
7. install pytorch 1.11.0+cu113 
    pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
8. CHECK
    nvidia-smi 
    >>> driver-version == 515, CUDA 11.7
    nvcc --version
    >>> CUDA 11.3.1
'''
# 꼭 CUDA & pytorch version 같은지 확인해야함.
import torch
print(torch.cuda.is_available())
print(torch.__version__)
# >>> Ture
# >>> 1.11.0+cu113
# VGA Driver의 CUDA와, CUDA Toolkit의 버전은 다를 수 있고, 전혀 상관 없다.
!nvidia-smi 
!nvcc --version

In [ ]:
%pip install preprocess
%pip install scikit-learn
%pip install scikit-image
%pip install Pillow==9.2.0

## 2.2 Download the dataset

KITTI/object/
    
    train.txt
    val.txt
    test.txt 
    
    training/
        calib/
        image_2/ #left image
        image_3/ #right image
        label_2/
        velodyne/ 

    testing/
        calib/
        image_2/
        image_3/
        velodyne/

http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d

https://github.com/charlesq34/frustum-pointnets/tree/master/kitti/image_sets

프로젝트/Pseudo_lidar-marter/KITTI/...

프로젝트/frustum_pointnet/...  의 형태로 디렉터리를 맞춘다.

## 2.3 Generate ground-truth image disparities


- velodyne의 .bin (point cloud data)를 .npy 로 바꿔줌.
- 실제 disparity를 구하는 부분

In [ ]:
'''
issue 1 : module error
    1. edit to './preprocessing/generate_disp.py'
        line 5 : import scipy.misc as ssc >> import skimage
        line 64 : ssc.imread >> skimage.io.imread
      
issue 2 : ubuntu system off
    1. add to './preprocessing/generate_disp.py' line 58-59;
        if predix < '______':
            continue
        ('______' is last saved disparity file. if last file is 001515.npy >> '_______' == 001500)
    2. save and retry
'''
!python3 ./preprocessing/generate_disp.py --data_path ./KITTI/object/training/ --split_file ./KITTI/object/train.txt 

## 2.4. Train the stereo model (Skip!!)


- KITTI Dataset을 이용해서 학습시킴.
- 오래걸리니, github에서 finetune_300.tar 파일을 다운받자.

In [ ]:
'''
if can't find module file
    1. $ gedit ~/.bashrc
    2. add to last line 
        export PYTHONPATH="${PYTHONPATH}:/home/(UESRNAME)/plv1/psmnet/models/"
    3. $ source ~/.bashrc
    4. reboot to vscode and THIS FILE
    5. CHECK
        import sys
        sys.path >> {..., '/home/(UESRNAME)/plv1/psmnet/models/', ...}

issue 2.
    /home/jiho/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: 
    UserWarning: This DataLoader will create 14 worker processes in total. 
    Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create.
    Please be aware that excessive worker creation might get DataLoader running slow or even freeze, 
    lower the worker number to avoid potential slowness/freeze if necessary.

    File "/home/jiho/.local/lib/python3.8/site-packages/torch/_utils.py", line 457, in reraise
      raise exception
    AttributeError: Caught AttributeError in DataLoader worker process 0.

    File "/home/jiho/plv1/psmnet/dataloader/KITTILoader_dataset3d.py", line 58, in __getitem__
      processed = preprocess.get_transform(augment=False)
    AttributeError: module 'preprocess' has no attribute 'get_transform'
    >>> add to python PATH

이거 해결 못했는데, 그냥 github 페이지 뒤지니까 3,712개 모델 학습된거 찾음. 그냥 파일 경로따라서 넣어주면됨
https://drive.google.com/file/d/1sWjsIO9Fuy92wT3gLkHF3PA7SP8QZBzu/view
./psmnet/kitti_3d/finetune_300.tar
'''
# 이거 우리 수준에서 절대 불가
# pseudo lidar github에 있는 psmnet 폴더에 있는 finetune_3d.py를 실행시켜야함. 현재의 finetune.py로는 불가
#!python3 ./psmnet/finetune_3d.py --maxdisp 192 --model stackhourglass --datapath ./KITTI/object/training/ --split_file ./KITTI/object/train.txt --epochs 300 --lr_scale 50 --loadmodel ./pretrained_sceneflow.tar --savemodel ./psmnet/kitti_3d/ --btrain 6

## 2.5.1 Predict the point clouds - Predict the disparities.


- training / .png >>> .npy(or .png)

In [3]:
'''
issue 1. 
    TypeError: new(): argument 'size' must be tuple of ints, but found element of type float at pos 3
    >>> float을 int로 형변환만 해주면 됨.

    "./psmnet/models/stackhourglass.py"
    torch.FloatTensor(refimg_fea.size()[0], refimg_fea.size()[1] * 2, self.maxdisp / 4, refimg_fea.size()[2], refimg_fea.size()[3]).zero_()).cuda()
    >>> torch.FloatTensor(refimg_fea.size()[0], refimg_fea.size()[1] * 2, int(self.maxdisp / 4), refimg_fea.size()[2], refimg_fea.size()[3]).zero_()).cuda()
    for i in range(self.maxdisp / 4):
    >>> for i in range(int(self.maxdisp / 4)):

issue 2. 
    RuntimeError: CUDA out of memory 부족 
    >>> 똥컴탓

issue 3.
    UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
    warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
    >>> upsample을 interpolate로 변경해주기.

finetune_300.tar 파일을 다운받고 ./psmnet에 넣어야 학습된 신경망 가져오기 가능.

두 psmnet 버전에 따라, input args가 다르다. 
1. 후 버전의 코드를 조금 수정해서, 변환 파일 저장 위치를 바꾼다.
2. submission.py의 output이 .png가 아니라 .npy가 되도록 기존버전에서 저장과 진행상황 알려주는 코드를 가져온다.
        np.save(args.save_path+'/'+test_left_img[inx].split('/')[-1][:-4], img)
        print(test_left_img[inx].split('/')[-1])
        >>> def main(): 의 맨 마지막에 코드 추가.
3. psmnet/dataloader/KITTI_submission_loader.py에서 line 24에 'image = sorted(image)'를 추가한다.
'''
%cd /home/jiho/plv1
!python3 ./psmnet/submission.py --loadmodel ./psmnet/finetune_300.tar --datapath ./KITTI/object/training/ --save_path ./KITTI/object/training/predict_disparity

/home/jiho/plv1
Number of model parameters: 5224768
time = 0.63
000000.png
time = 0.49
000001.png
time = 0.50
000002.png
time = 0.49
000003.png
time = 0.48
000004.png
^C
Traceback (most recent call last):
  File "./psmnet/submission.py", line 116, in <module>
    main()
  File "./psmnet/submission.py", line 104, in main
    pred_disp = test(imgL,imgR)
  File "./psmnet/submission.py", line 70, in test
    output = model(imgL,imgR)
  File "/home/jiho/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/jiho/.local/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py", line 166, in forward
    return self.module(*inputs[0], **kwargs[0])
  File "/home/jiho/.local/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/jiho/plv1/psmnet/models/stackhourglass.py", line 110, in forward
    cost = Variable(torch.F

- testing / .png >>> .npy(or .png)

In [ ]:
!python3 ./psmnet/submission.py --loadmodel ./psmnet/kitti_3d/finetune_300.tar --datapath ./KITTI/object/testing/ --save_path ./KITTI/object/testing/predict_disparity

## 2.5.2 Predict the point clouds - Convert the disparities to point clouds.

- training / .npy(or .png) >>> .bin

In [7]:
!python3 ./preprocessing/generate_lidar.py --calib_dir ./KITTI/object/training/calib/ --save_dir ./KITTI/object/training/pseudo-lidar_velodyne/ --disparity_dir ./KITTI/object/training/predict_disparity --max_high 1

Finish Depth 000000
Finish Depth 000001
Finish Depth 000002
Finish Depth 000003
Finish Depth 000004


- testing  / .npy(or .png) >>> .bin


In [ ]:
!python3 ./preprocessing/generate_lidar.py --calib_dir ./KITTI/object/testing/calib/ --save_dir ./KITTI/object/testing/pseudo-lidar_velodyne/ --disparity_dir ./KITTI/object/testing/predict_disparity --max_high 1

- 2.5.1과 2.5.2의 과정을 거치면 .png >>> .npy >>> .bin 파일이 된다.
- 이 파일들은 바로 밑에서 path를 수정하고 실행해서 확인

In [8]:
from pyntcloud import PyntCloud
import pandas as pd
import os
import numpy as np
import PIL.Image as Image
%matplotlib inline
def load_velo_scan(velo_filename):
    scan = np.fromfile(velo_filename, dtype=np.float32)
    scan = scan.reshape((-1, 4))
    return scan
def paint_points(points, color=[192,0,0]):
    # color = [r, g, b]
    color = np.array([color])
    new_pts = np.zeros([points.shape[0],6])
    new_pts[:,:3] = points
    new_pts[:, 3:] = new_pts[:, 3:] + color
    return new_pts
# 여기를 보고싶은 .bin 파일의 경로로 수정.
path = '/home/jiho/plv1/KITTI/object/training/pseudo-lidar_velodyne/000000.bin'
points = load_velo_scan(path)[:,:3]
pd_points = pd.DataFrame(paint_points(points), columns=['x','y','z','red','green','blue'])
cloud = PyntCloud(pd_points)
cloud.plot(initial_point_size=0.02)

/home/jiho/.local/lib/python3.8/site-packages/pythreejs/traits.py:203: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '
/home/jiho/.local/lib/python3.8/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(11.347519311071496, 18.742071965291153, 0.62…

## 2.6 Generate ground plane

- training

In [1]:
!python3 ./preprocessing/kitti_process_RANSAC.py --calib ./KITTI/object/training/calib/ --lidar_dir  ./KITTI/object/training/pseudo-lidar_velodyne/ --planes_dir ./KITTI/object/training/pseudo-lidar_planes/

-------------  000000
[[ 0.00692796 -0.9999722  -0.00275783]
 [-0.00116298  0.00274984 -0.9999955 ]
 [ 0.9999753   0.00693114 -0.0011439 ]
 [ 0.33219371 -0.02210627 -0.06171977]]
12
[-0.00810649 -0.9997002  -0.02310421]
1.6868425923156682
-------------  000004
[[ 7.53374500e-03 -9.99971400e-01 -6.16602000e-04]
 [ 1.48024900e-02  7.28073300e-04 -9.99890200e-01]
 [ 9.99862100e-01  7.52379000e-03  1.48075500e-02]
 [ 2.72903452e-01 -1.96926567e-03 -7.22859051e-02]]
12
[-0.00168079 -0.99999426  0.00294331]
1.6519988373158707
-------------  000003
[[ 7.53374500e-03 -9.99971400e-01 -6.16602000e-04]
 [ 1.48024900e-02  7.28073300e-04 -9.99890200e-01]
 [ 9.99862100e-01  7.52379000e-03  1.48075500e-02]
 [ 2.72903452e-01 -1.96926567e-03 -7.22859051e-02]]
12
[-0.06423007 -0.99774134  0.01966527]
1.5695283375878053
-------------  000001
[[ 7.53374500e-03 -9.99971400e-01 -6.16602000e-04]
 [ 1.48024900e-02  7.28073300e-04 -9.99890200e-01]
 [ 9.99862100e-01  7.52379000e-03  1.48075500e-02]
 [ 2.7290345

- testing

In [ ]:
!python3 ./preprocessing/kitti_process_RANSAC.py --calib ./KITTI/object/testing/calib/ --lidar_dir  ./KITTI/object/testing/pseudo-lidar_velodyne/ --planes_dir ./KITTI/object/testing/pseudo-lidar_planes/

# 3. Object Detection models - Frustum-PointNets model

pip install mayavi
pip install opencv-python
sudo apt install libxcb-xinerama0-dev
pip install pyqt5

## Usage

### Setting Check - 객체 라벨링 확인할 수 있음

In [10]:
'''
여기서부터는 frustum_pointnet dir로 이동해서 실행
issue 1.
    File "/home/jiho/my/kitti/kitti_util.py", line 381, in draw_projected_box3d
    cv2.line(image, (qs[i,0],qs[i,1]), (qs[j,0],qs[j,1]), color, thickness, cv2.CV_AA)
    solution : CV_AA >>> LINE_AA

한 쌍의 .png로부터 얻은 모든 정보를 볼 수 있다.
'''
%cd /home/jiho/my
!python3 kitti/kitti_object.py

/home/jiho/my
multiple 'pyface.toolkits' plugins found for toolkit 'qt4': pyface.ui.qt4.init, pyface.ui.qt4.init
^C
Traceback (most recent call last):
  File "kitti/kitti_object.py", line 222, in <module>
    import mayavi.mlab as mlab
  File "/usr/local/lib/python3.8/dist-packages/mayavi/mlab.py", line 34, in <module>
    from .tools.helper_functions import contour3d, test_contour3d, \
  File "/home/jiho/.local/lib/python3.8/site-packages/shiboken2/files.dir/shibokensupport/__feature__.py", line 142, in _import
    return original_import(name, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/mayavi/tools/helper_functions.py", line 16, in <module>
    from .modules import VectorsFactory, StreamlineFactory, GlyphFactory, \
  File "/home/jiho/.local/lib/python3.8/site-packages/shiboken2/files.dir/shibokensupport/__feature__.py", line 142, in _import
    return original_import(name, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/mayavi/tools/modules.py", li

### Prepare Training Data


- .bin, .png 를 이용하여, python에서 읽을 수 있는 pickle file로 만든다.

In [ ]:
'''
issue 1.
  File "kitti/prepare_data.py", line 18, in <module>
  import cPickle as pickle
  >>> import pickle

but, 4.7GB짜리 .plckle file set 옮겨놓으면 생략가능.

issue 2.
  prepare_data.py; line 46.
  from viz_util import *
  >>> add PATH /home/jiho/my/mayavi

애초에 pseudo lidar가 올바르게 만들어지지 않아서, pickle의 데이터 용량도 작았던 것임.
라이다 제대로 변환해서, 피클만 똑바로 뽑아내면 이후 확인작업 수월히 할 수 있음.
pickle 데이터는 lidar에 영향받음.

원래 pseudo_lidar-velodyne 폴더를 velodyne으로 이름 변경해야한다.
테스트 시, 원본 KITTI dataset과 섞이지 않게 주의.
'''
%cd /home/jiho/my
#!sh scripts/command_prep_data.sh
!python3 kitti/prepare_data.py --gen_train
#!python3 kitti/prepare_data.py --gen_val
#!python3 kitti/prepare_data.py --gen_val_rgb_detection


In [ ]:
'''
- frustum_carpedcyc_train_detection.pickle의 감지된 객체 2만 여개 항목 틀어줌.
- 터미널에서 실행 후, 결과 나온 뒤 엔터 누르면 다음 출력으로 넘어감.
- 2d, 3d, pseudo lidar 등 여러 정보 확인 가능.
'''
!python3 ./train/provider.py

In [ ]:
!python3 ./kitti/prepare_data.py --demo

출력 불가 시 : pyqt5 9.x버전쓰면안됨. 개발자들이 버전 7에서 삽질함. 6.x로 깔면됨

이 부분 여기 정확하지 않아서 수정예정

### Frustum PointNet 훈련


- Frustum Pointnet의 딥러닝 과정.
- 이미 훈련 시킨 모델파일 제공함

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 sh scripts/command_train_v1.sh
#!CUDA_VISIBLE_DEVICES=0 python3 train/train.py --gpu 0 --model frustum_pointnets_v1 --log_dir train/log_v1 --num_point 1024 --max_epoch 201 --batch_size 32 --decay_step 800000 --decay_rate 0.5

NEW: We have also prepared some pretrained snapshots for both the v1 and v2 models. You can find them HERE (40MB) -- to support evaluation script, you just need to unzip the file and move the log_* folders to the train folder.

frustum github에서 파일받고, log_v1, log_v2 총 40MB 옮기면 됨.

### 평가
- Frustum pointnet 알고리즘이 차량, 보행자, 자전거 등을 얼마나 감지했는지 통계냄.
- pickle file을 바탕으로 통계를 작성하기 때문에, pickle file 생성에 예민하다.

In [ ]:
'''
issue 1.
    train/test.py
    line 16 
    import cPickle as pickle
    >> import pickle
issue 2.
    PATH : model_util.py 
    
issue 3.
 File "train/test.py", line 231
    batch_output, batch_center_pred, \
                                     ^
TabError: inconsistent use of tabs and spaces in indentation

issue 4.
 File "/home/jiho/my/train/provider.py", line 8, in <module>
    import cPickle as pickle
    
issue 5.
Traceback (most recent call last):
  File "train/test.py", line 48, in <module>
    TEST_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='val',
  File "/home/jiho/my/train/provider.py", line 129, in __init__
    self.box2d_list = pickle.load(fp)
    UnicodeDecodeError: 'ascii' codec can't decode byte 0x85 in position 6: ordinal not in range(128)
>> 피클파일 직접 만들기

issue 6. ===============텐서 수정
  tf.placeholder 작동안함
  >> test.py, model_uilt.py 안에서 모두 tf.compat.v1.placeholder로 변경

issue 7. ===============텐서 수정
  File "/home/jiho/my/models/frustum_pointnets_v1.py", line 34, in get_instance_seg_v1_net
    batch_size = point_cloud.get_shape()[0].value
AttributeError: 'int' object has no attribute 'value'

34,35 줄     batch_size = point_cloud.get_shape()[0]
    num_point = point_cloud.get_shape()[1]
    그냥 .value 지워버림

issue 8.    
models/tf_util.py; line 155
with tf.compat.v1.variable_scope(scope) as sc:

9.
File "/home/jiho/my/models/tf_util.py", line 527, in batch_norm_template
    return tf.contrib.layers.batch_norm(inputs, 
AttributeError: module 'tensorflow' has no attribute 'contrib'

    return tf.layers.batch_norm(inputs, 

!pip install tf-slim
import tf_slim as slim
  return slim.batch_norm(inputs, 
으로 변경

10.
packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/jiho/my/models/tf_util.py", line 614, in <lambda>
    true_fn=lambda: tf.nn.dropout(inputs, noise_shape, rate=1 - (keep_prob)),
TypeError: Got multiple values for argument 'rate'

tensor 2로 올리면서 코드가 꼬임. 원본을 가져다 붙임.
    outputs = tf.cond(is_training,
                      lambda: tf.nn.dropout(inputs, keep_prob, noise_shape),
                      lambda: inputs)
11.
model_util.py 에서 value 삭제

12.
2022-08-07 19:04:21.478824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] 
Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: 
No such file or directory; 

install cudnn 
Local Installer for Ubuntu20.04 x86_64 (Deb)
https://developer.nvidia.com/rdp/cudnn-download

13.
model_util.py; line65
  indices = tf.py_function(mask_to_indices, [mask], tf.int32)  
  텐서 버전에 맞는 문법으로 고치기

14. 
batch idx: 792
batch idx: 793
Traceback (most recent call last):
  File "train/test.py", line 355, in <module>
    test_from_rgb_detection(FLAGS.output+'.pickle', FLAGS.output)
  File "train/test.py", line 234, in test_from_rgb_detection
    inference(sess, ops, batch_data_to_feed,
  File "train/test.py", line 113, in inference
    for i in range(num_batches):
TypeError: 'float' object cannot be interpreted as an integer
>>>     for i in range(int(num_batches)):

15. 
  File "train/test.py", line 114, in inference
    ops['pointclouds_pl']: pc[i*batch_size:(i+1)*batch_size,...],
TypeError: slice indices must be integers or None or have an __index__ method

    num_batches = pc.shape[0]/batch_size
>>>     num_batches = int(pc.shape[0]/batch_size)

16. install cudnn for CUDA 11.3 in ubuntu 20.04
  https://lapina.tistory.com/137

  successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
  !cat /sys/bus/pci/devices/0000\:01\:00.0/numa_node
  >>> -1
  !echo 0 | sudo tee -a /sys/bus/pci/devices/0000\:01\:00.0/numa_node
  >>> 0
  !cat /sys/bus/pci/devices/0000\:01\:00.0/numa_node
  >>> 0

17. edit batch size
  under 16(deflaut == 32)

18. provider.py
    싹 다 바꾸기
    self.~~~~ = pickle.load(fp)
    >>> self.~~~~ = pickle.load(fp,encoding = 'ISO-8859-1')
'''
#%rm -r ./train/detection_results_v1/plot
#!CUDA_VISIBLE_DEVICES=0 sh scripts/command_test_v1.sh
!CUDA_VISIBLE_DEVICES=0 python3 train/test.py --gpu 0 --num_point 1024 --model_path train/log_v1/model.ckpt --batch_size 32 --output train/detection_results_v1 --data_path kitti/frustum_carpedcyc_val_rgb_detection.pickle --from_rgb_detection --idx_path kitti/image_sets/val.txt --from_rgb_detection

In [ ]:
'''
apt-get install texlive-extra-utils
apt-get install gnuplot
apt-get install ghostscript

'''
!train/kitti_eval/evaluate_object_3d_offline dataset/KITTI/object/training/label_2/ train/detection_results_v1

### Prepare Training 
- frustum pointnet의 탐지 결과에 관한 readme 파일

In this step we convert original KITTI data to organized formats for training our Frustum PointNets. <b>NEW:</b> You can also directly download the prepared data files <a href="https://shapenet.cs.stanford.edu/media/frustum_data.zip" target="_blank">HERE (960MB)</a> -- to support training and evaluation, just unzip the file and move the `*.pickle` files to the `kitti` folder.

Firstly, you need to download the <a href="http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d" target="_blank">KITTI 3D object detection dataset</a>, including left color images, Velodyne point clouds, camera calibration matrices, and training labels. Make sure the KITTI data is organized as required in `dataset/README.md`. You can run `python kitti/kitti_object.py` to see whether data is downloaded and stored properly. If everything is fine, you should see image and 3D point cloud visualizations of the data. 

Then to prepare the data, simply run: (warning: this step will generate around 4.7GB data as pickle files)

    sh scripts/command_prep_data.sh

Basically, during this process, we are extracting frustum point clouds along with ground truth labels from the original KITTI data, based on both ground truth 2D bounding boxes and boxes from a 2D object detector. We will do the extraction for the train (`kitti/image_sets/train.txt`) and validation set (`kitti/image_sets/val.txt`) using ground truth 2D boxes, and also extract data from validation set with predicted 2D boxes (`kitti/rgb_detections/rgb_detection_val.txt`).

You can check `kitti/prepare_data.py` for more details, and run `python kitti/prepare_data.py --demo` to visualize the steps in data preparation.

After the command executes, you should see three newly generated data files under the `kitti` folder. You can run `python train/provider.py` to visualize the training data (frustum point clouds and 3D bounding box labels, in rect camera coordinate).